In [1]:
import os
import boto3
from dotenv import load_dotenv, set_key

ENV_PATH = ".env"

# Load .env if it exists
load_dotenv(ENV_PATH)

bucket = os.getenv("BUCKET_NAME")

session = boto3.session.Session()
region = session.region_name
s3 = boto3.client("s3", region_name=region)

# If BUCKET_NAME exists, use it
if bucket:
    print("Using existing bucket from .env:", bucket)

else:
    # Create new bucket name
    print("Run load_data.ipynb to create a new S3 bucket and load data into it.")
    raise Exception("BUCKET_NAME not found in .env")

Using existing bucket from .env: mlops-backblaze-d7b30cb5-us-east-1


## Read Backblaze Parquet Files with AWS Glue

Use AWS Glue Interactive Sessions to read the partitioned parquet files with a managed Spark environment.

In [2]:
# Install and configure AWS Glue Sessions
# Note: For Glue Interactive Sessions, you need to use the Glue kernel or %%configure magic
# Here we'll use the boto3 approach to work with Glue Data Catalog

# Initialize Glue client
glue_client = boto3.client('glue', region_name=region)

print("AWS Glue client initialized")
print(f"Region: {region}")
print(f"\nGlue can be used to:")
print("  1. Query the Athena table we created")
print("  2. Use Glue Data Catalog for schema management")
print("  3. Run Glue ETL jobs for transformations")
print("  4. Use Glue Interactive Sessions (requires Glue kernel)")

AWS Glue client initialized
Region: us-east-1

Glue can be used to:
  1. Query the Athena table we created
  2. Use Glue Data Catalog for schema management
  3. Run Glue ETL jobs for transformations
  4. Use Glue Interactive Sessions (requires Glue kernel)


In [3]:
# Read Backblaze parquet files using PyArrow and S3
# This is a lightweight alternative to Spark when Glue Interactive Sessions aren't available

import pyarrow.parquet as pq
import pyarrow.dataset as ds
import pyarrow.compute as pc

print("Reading Backblaze parquet dataset from S3 using PyArrow...")
print(f"S3 Path: s3://{bucket}/curated/backblaze_parquet/\n")

# Create dataset from S3 path
# PyArrow can efficiently read partitioned parquet files
dataset = ds.dataset(
    f"s3://{bucket}/curated/backblaze_parquet/",
    format="parquet",
    partitioning="hive"  # Recognizes year=/month=/day= partitioning
)

print(f"Dataset loaded!")
print(f"\nDataset schema (first 30 fields):")
schema_fields = list(dataset.schema)
for i, field in enumerate(schema_fields[:30], 1):
    print(f"  {i}. {field.name}: {field.type}")

if len(schema_fields) > 30:
    print(f"\n  ... and {len(schema_fields) - 30} more fields")

print(f"\nTotal fields: {len(schema_fields)}")
print(f"Partitioning: {dataset.partitioning}")
print(f"Format: {dataset.format}")

Reading Backblaze parquet dataset from S3 using PyArrow...
S3 Path: s3://mlops-backblaze-d7b30cb5-us-east-1/curated/backblaze_parquet/

Dataset loaded!

Dataset schema (first 30 fields):
  1. date: large_string
  2. serial_number: large_string
  3. model: large_string
  4. capacity_bytes: int64
  5. failure: int64
  6. datacenter: large_string
  7. cluster_id: int64
  8. vault_id: int64
  9. pod_id: int64
  10. pod_slot_num: double
  11. is_legacy_format: bool
  12. smart_1_normalized: double
  13. smart_1_raw: double
  14. smart_2_normalized: double
  15. smart_2_raw: double
  16. smart_3_normalized: double
  17. smart_3_raw: double
  18. smart_4_normalized: double
  19. smart_4_raw: double
  20. smart_5_normalized: double
  21. smart_5_raw: double
  22. smart_7_normalized: double
  23. smart_7_raw: double
  24. smart_8_normalized: double
  25. smart_8_raw: double
  26. smart_9_normalized: double
  27. smart_9_raw: double
  28. smart_10_normalized: double
  29. smart_10_raw: double
  

In [4]:
# Read dataset into a PyArrow Table
# For large datasets, use column selection and filtering

print("Reading data into PyArrow Table...")

# ============================================================================
# PARAMETER: Number of days to read from parquet files
# ============================================================================
# Controls how many days of historical data to load from the partitioned parquet files
# - Lower values = faster loading, less data, more recent data only
# - Higher values = slower loading, more data, broader historical coverage
# Default: 5 days (last 5 days of available data)
NUM_DAYS_TO_READ = 5
# ============================================================================

print(f"Reading the last {NUM_DAYS_TO_READ} days of data...\n")

# Get all unique days by scanning partition information
import datetime

# Get fragments to identify unique dates
all_partitions = []
for fragment in dataset.get_fragments():
    partition_expr = fragment.partition_expression
    # Extract year, month, day from partition expression
    partition_str = str(partition_expr)
    all_partitions.append(partition_str)

# Get unique partition combinations and sort them
unique_partitions = sorted(set(all_partitions))

print(f"Total unique date partitions available: {len(unique_partitions)}")
print(f"Last few partitions: {unique_partitions[-min(5, len(unique_partitions)):]}\n")

# Parse last N partitions to build date filter
selected_days = []
for partition in unique_partitions[-(NUM_DAYS_TO_READ * 3):]:  # Get extra in case some partitions aren't dates
    # Extract year, month, day values from partition string
    # Format: "(((year == 2025) and (month == 01)) and (day == 01))"
    try:
        if 'year' in partition and 'month' in partition and 'day' in partition:
            year_str = partition.split('year == ')[1].split(')')[0].strip().strip("'\"")
            month_str = partition.split('month == ')[1].split(')')[0].strip().strip("'\"")
            day_str = partition.split('day == ')[1].split(')')[0].strip().strip("'\"")
            
            # Convert to integers for proper comparison
            year = int(year_str)
            month = int(month_str)
            day = int(day_str)
            
            selected_days.append((year, month, day))
            if len(selected_days) >= NUM_DAYS_TO_READ:
                break
    except:
        continue

print(f"Selected {len(selected_days)} days:")
for year, month, day in selected_days:
    print(f"  - {year:04d}-{month:02d}-{day:02d}")

# Build filter expression for selected days
if selected_days:
    # Create OR filter for each day
    day_filters = []
    for year, month, day in selected_days:
        day_filter = (
            (pc.field('year') == year) & 
            (pc.field('month') == month) & 
            (pc.field('day') == day)
        )
        day_filters.append(day_filter)
    
    # Combine all day filters with OR
    combined_filter = day_filters[0]
    for day_filter in day_filters[1:]:
        combined_filter = combined_filter | day_filter
    
    # Read data with filter - reading ALL columns
    sample_table = dataset.to_table(
        filter=combined_filter
    )
else:
    # Fallback: just read with a limit
    sample_table = dataset.to_table().slice(0, 10000)

print(f"\nData loaded from last {len(selected_days)} days!")
print(f"Rows: {sample_table.num_rows:,}")
print(f"Columns: {sample_table.num_columns}")
print(f"Size in memory: ~{sample_table.nbytes / 1024 / 1024:.1f} MB")

print(f"\nFirst 5 rows (showing ALL {sample_table.num_columns} columns):")
print(sample_table.slice(0, 5).to_pandas())


Reading data into PyArrow Table...
Reading the last 5 days of data...

Total unique date partitions available: 273
Last few partitions: ['(((year == 2025) and (month == 9)) and (day == 5))', '(((year == 2025) and (month == 9)) and (day == 6))', '(((year == 2025) and (month == 9)) and (day == 7))', '(((year == 2025) and (month == 9)) and (day == 8))', '(((year == 2025) and (month == 9)) and (day == 9))']

Selected 5 days:
  - 2025-09-23
  - 2025-09-24
  - 2025-09-25
  - 2025-09-26
  - 2025-09-27

Data loaded from last 5 days!
Rows: 1,645,930
Columns: 200
Size in memory: ~2581.4 MB

First 5 rows (showing ALL 200 columns):
         date serial_number           model  capacity_bytes  failure  \
0  2025-09-23  2206E608DB42  CT250MX500SSD1    250059350016        0   
1  2025-09-23  2207E60CC65A  CT250MX500SSD1    250059350016        0   
2  2025-09-23  2340E87B92B5  CT250MX500SSD1    250059350016        0   
3  2025-09-23  2340E87B97E8  CT250MX500SSD1    250059350016        0   
4  2025-09-2

In [5]:
# Convert to Pandas for analysis
import pandas as pd

df_pandas = sample_table.to_pandas()

print(f"Converted to Pandas DataFrame")
print(f"Shape: {df_pandas.shape}")
print(f"\nData types:")
print(df_pandas.dtypes)
print(f"\nFirst few rows:")
print(df_pandas.head())


Converted to Pandas DataFrame
Shape: (1645930, 200)

Data types:
date                     object
serial_number            object
model                    object
capacity_bytes            int64
failure                   int64
                         ...   
smart_255_normalized    float64
smart_255_raw           float64
year                      int32
month                     int32
day                       int32
Length: 200, dtype: object

First few rows:
         date serial_number           model  capacity_bytes  failure  \
0  2025-09-23  2206E608DB42  CT250MX500SSD1    250059350016        0   
1  2025-09-23  2207E60CC65A  CT250MX500SSD1    250059350016        0   
2  2025-09-23  2340E87B92B5  CT250MX500SSD1    250059350016        0   
3  2025-09-23  2340E87B97E8  CT250MX500SSD1    250059350016        0   
4  2025-09-23  2407E896B6D5  CT250MX500SSD1    250059350016        0   

  datacenter  cluster_id  vault_id  pod_id  pod_slot_num  ...  smart_251_raw  \
0       sac0           0  

## PyArrow Dataset Analysis Examples

Perform analysis on the Backblaze data using PyArrow and Pandas.

In [6]:
# Example 1: Analyze failure rates by manufacturer using Pandas
print("Calculating failure rates by manufacturer...\n")

# Extract manufacturer from model
df_pandas['manufacturer'] = df_pandas['model'].str.split(' ').str[0]

# Group and calculate statistics
manufacturer_stats = df_pandas.groupby('manufacturer').agg({
    'failure': ['count', 'sum']
})

manufacturer_stats.columns = ['total_records', 'total_failures']
manufacturer_stats['failure_rate_pct'] = (
    manufacturer_stats['total_failures'] / manufacturer_stats['total_records'] * 100
).round(2)

manufacturer_stats = manufacturer_stats.sort_values('total_records', ascending=False)

print("Failure rates by manufacturer (January 2025):")
print(manufacturer_stats.head(10))

Calculating failure rates by manufacturer...

Failure rates by manufacturer (January 2025):
               total_records  total_failures  failure_rate_pct
manufacturer                                                  
TOSHIBA               549909              50              0.01
WDC                   383069              11              0.00
ST16000NM001G         170239               4              0.00
HGST                  136048              27              0.02
ST12000NM0008          93507               9              0.01
ST8000NM0055           66427              10              0.02
ST12000NM001G          66237               7              0.01
ST14000NM001G          52797               3              0.01
ST8000DM002            44732               1              0.00
ST24000NM002H          36002               2              0.01


## Additional Glue Operations

### Convert to Pandas (for small datasets)
```python
# Only convert small subsets to Pandas to avoid memory issues
pandas_df = df_spark.limit(1000).toPandas()
```

### Write processed data back to S3
```python
# Write as Parquet
df_spark.write.mode("overwrite").parquet("s3://bucket/path/output/")

# Write as CSV
df_spark.write.mode("overwrite").option("header", "true").csv("s3://bucket/path/output/")
```

### Use Glue Data Catalog
```python
# Write to Glue Data Catalog table
glueContext.write_dynamic_frame.from_catalog(
    frame=datasource,
    database="backblaze_db",
    table_name="drive_stats"
)
```

### Partition output data
```python
# Write with partitioning
df_spark.write.mode("overwrite") \
    .partitionBy("year", "month") \
    .parquet("s3://bucket/path/output/")
```

## Complete Column Information

Display detailed information about all columns in the Backblaze dataset including data types, descriptions, and sample values.

In [7]:
# Get complete schema information using PyArrow
import pandas as pd

print("="*80)
print("COMPLETE COLUMN INFORMATION - BACKBLAZE DATASET")
print("="*80)

schema_fields = list(dataset.schema)
print(f"\nTotal Columns: {len(schema_fields)}\n")

# Create a detailed column info dataframe
column_info = []
for i, field in enumerate(schema_fields, 1):
    column_info.append({
        'Index': i,
        'Column Name': field.name,
        'Data Type': str(field.type),
        'Nullable': field.nullable
    })

df_columns = pd.DataFrame(column_info)

# Display all columns
print("All Columns with Data Types:")
print("-"*80)
pd.set_option('display.max_rows', None)
print(df_columns.to_string(index=False))
pd.reset_option('display.max_rows')

COMPLETE COLUMN INFORMATION - BACKBLAZE DATASET

Total Columns: 200

All Columns with Data Types:
--------------------------------------------------------------------------------
 Index          Column Name    Data Type  Nullable
     1                 date large_string      True
     2        serial_number large_string      True
     3                model large_string      True
     4       capacity_bytes        int64      True
     5              failure        int64      True
     6           datacenter large_string      True
     7           cluster_id        int64      True
     8             vault_id        int64      True
     9               pod_id        int64      True
    10         pod_slot_num       double      True
    11     is_legacy_format         bool      True
    12   smart_1_normalized       double      True
    13          smart_1_raw       double      True
    14   smart_2_normalized       double      True
    15          smart_2_raw       double      True
    1

In [8]:
# Categorize columns by type
print("\n" + "="*80)
print("COLUMNS BY CATEGORY")
print("="*80)

# Core identification columns
core_cols = [f for f in schema_fields if f.name in ['date', 'serial_number', 'model', 'capacity_bytes', 'failure']]
print(f"\n1. CORE DRIVE IDENTIFICATION ({len(core_cols)} columns):")
for field in core_cols:
    print(f"   • {field.name}: {field.type}")

# Location/datacenter columns
location_cols = [f for f in schema_fields if f.name in ['datacenter', 'cluster_id', 'vault_id', 'pod_id', 'pod_slot_num']]
print(f"\n2. DATACENTER LOCATION ({len(location_cols)} columns):")
for field in location_cols:
    print(f"   • {field.name}: {field.type}")

# Partition columns
partition_cols = [f for f in schema_fields if f.name in ['year', 'month', 'day']]
print(f"\n3. PARTITION KEYS ({len(partition_cols)} columns):")
for field in partition_cols:
    print(f"   • {field.name}: {field.type}")

# SMART attribute columns
smart_cols = [f for f in schema_fields if 'smart' in f.name.lower()]
print(f"\n4. SMART ATTRIBUTES ({len(smart_cols)} columns):")
print(f"   These are hard drive health monitoring attributes")
print(f"   Format: smart_X_normalized and smart_X_raw")
print(f"\n   Common SMART attributes include:")

# Group SMART attributes by ID
smart_ids = set()
for field in smart_cols:
    if 'smart_' in field.name:
        parts = field.name.split('_')
        if len(parts) >= 2 and parts[1].isdigit():
            smart_ids.add(int(parts[1]))

smart_ids = sorted(list(smart_ids))
print(f"   SMART IDs present: {smart_ids[:20]}...")
print(f"   Total unique SMART IDs: {len(smart_ids)}")


COLUMNS BY CATEGORY

1. CORE DRIVE IDENTIFICATION (5 columns):
   • date: large_string
   • serial_number: large_string
   • model: large_string
   • capacity_bytes: int64
   • failure: int64

2. DATACENTER LOCATION (5 columns):
   • datacenter: large_string
   • cluster_id: int64
   • vault_id: int64
   • pod_id: int64
   • pod_slot_num: double

3. PARTITION KEYS (3 columns):
   • year: int32
   • month: int32
   • day: int32

4. SMART ATTRIBUTES (186 columns):
   These are hard drive health monitoring attributes
   Format: smart_X_normalized and smart_X_raw

   Common SMART attributes include:
   SMART IDs present: [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 22, 23, 24, 27]...
   Total unique SMART IDs: 93


In [9]:
# Detailed SMART attribute descriptions
print("\n" + "="*80)
print("KEY SMART ATTRIBUTES - DETAILED DESCRIPTIONS")
print("="*80)

smart_descriptions = {
    1: "Read Error Rate - Rate of hardware read errors",
    2: "Throughput Performance - Overall throughput performance",
    3: "Spin-Up Time - Time to spin up the drive",
    4: "Start/Stop Count - Number of drive start/stop cycles",
    5: "Reallocated Sectors Count - Count of reallocated sectors (CRITICAL)",
    7: "Seek Error Rate - Rate of seek errors",
    8: "Seek Time Performance - Average seek time performance",
    9: "Power-On Hours - Total hours drive has been powered on",
    10: "Spin Retry Count - Number of retry attempts to spin up",
    11: "Calibration Retry Count - Number of calibration retries",
    12: "Power Cycle Count - Number of power-on/off cycles",
    183: "Runtime Bad Block - Runtime bad block count",
    184: "End-to-End Error - End-to-end error detection count",
    187: "Reported Uncorrectable Errors - Errors that could not be corrected (CRITICAL)",
    188: "Command Timeout - Count of command timeout events",
    189: "High Fly Writes - Head flying height errors",
    190: "Temperature - Drive temperature in Celsius",
    191: "G-Sense Error Rate - Rate of errors from shock/vibration",
    192: "Power-Off Retract Count - Emergency head retracts",
    193: "Load/Unload Cycle Count - Number of head load/unload cycles",
    194: "Temperature Celsius - Current drive temperature",
    195: "Hardware ECC Recovered - Errors corrected by ECC",
    196: "Reallocation Event Count - Count of remap operations",
    197: "Current Pending Sector Count - Sectors waiting to be remapped (CRITICAL)",
    198: "Offline Uncorrectable - Uncorrectable sector count (CRITICAL)",
    199: "UltraDMA CRC Error Count - CRC errors during data transfer",
    200: "Multi-Zone Error Rate - Write error rate",
    220: "Disk Shift - Drive shift from intended position",
    222: "Loaded Hours - Time spent with heads loaded",
    223: "Load/Unload Retry Count - Head load/unload retry attempts",
    224: "Load Friction - Friction during head load",
    225: "Load/Unload Cycle Count - Total head load cycles",
    226: "Load In Time - Time to load heads",
    240: "Head Flying Hours - Time spent with heads flying",
    241: "Total LBAs Written - Lifetime total data written",
    242: "Total LBAs Read - Lifetime total data read"
}

print("\nCRITICAL FAILURE INDICATORS:")
print("-"*80)
critical_smart = [5, 187, 197, 198]
for smart_id in critical_smart:
    if smart_id in smart_descriptions:
        print(f"SMART {smart_id:3d}: {smart_descriptions[smart_id]}")

print("\nCOMMONLY MONITORED ATTRIBUTES:")
print("-"*80)
common_smart = [1, 3, 4, 7, 9, 12, 190, 194, 199, 241, 242]
for smart_id in common_smart:
    if smart_id in smart_descriptions:
        print(f"SMART {smart_id:3d}: {smart_descriptions[smart_id]}")

print(f"\nNote: Each SMART attribute has two values:")
print(f"  • smart_X_normalized: Normalized value (0-100, higher is usually better)")
print(f"  • smart_X_raw: Raw value (actual count/measurement)")


KEY SMART ATTRIBUTES - DETAILED DESCRIPTIONS

CRITICAL FAILURE INDICATORS:
--------------------------------------------------------------------------------
SMART   5: Reallocated Sectors Count - Count of reallocated sectors (CRITICAL)
SMART 187: Reported Uncorrectable Errors - Errors that could not be corrected (CRITICAL)
SMART 197: Current Pending Sector Count - Sectors waiting to be remapped (CRITICAL)
SMART 198: Offline Uncorrectable - Uncorrectable sector count (CRITICAL)

COMMONLY MONITORED ATTRIBUTES:
--------------------------------------------------------------------------------
SMART   1: Read Error Rate - Rate of hardware read errors
SMART   3: Spin-Up Time - Time to spin up the drive
SMART   4: Start/Stop Count - Number of drive start/stop cycles
SMART   7: Seek Error Rate - Rate of seek errors
SMART   9: Power-On Hours - Total hours drive has been powered on
SMART  12: Power Cycle Count - Number of power-on/off cycles
SMART 190: Temperature - Drive temperature in Celsius
S

## Read All Review Parquet Files into DataFrame

In [10]:
import glob
import pandas as pd
import os

# Get all parquet files from the reviews parquet directory
parquet_dir = "amazon_reviews_download/parquet/Electronics"
parquet_pattern = os.path.join(parquet_dir, "**/*.parquet")
all_parquet_files = glob.glob(parquet_pattern, recursive=True)

print(f"Found {len(all_parquet_files)} parquet files")
if all_parquet_files:
    print(f"First few files: {all_parquet_files[:5]}")
    
    # Read all parquet files into a single dataframe
    df_all_reviews = pd.concat([pd.read_parquet(f) for f in all_parquet_files], ignore_index=True)
    
    print(f"\nDataFrame shape: {df_all_reviews.shape}")
    print(f"Columns: {list(df_all_reviews.columns)}")
    print(f"\nFirst few rows:")
    print(df_all_reviews.head())
else:
    print("No parquet files found. Checking if files exist from S3...")
    # If no local files, use the reviews_parquet_files list from S3
    if 'reviews_parquet_files' in dir() and reviews_parquet_files:
        print(f"Found {len(reviews_parquet_files)} parquet files in S3")
        print(f"First few files: {reviews_parquet_files[:5]}")

Found 0 parquet files
No parquet files found. Checking if files exist from S3...


In [18]:
# Read all parquet files from S3
df_all_reviews_s3 = pd.DataFrame()

# List parquet files from S3
reviews_prefix = "raw/reviews_2023_parquet/raw_review_Electronics/"
response = s3.list_objects_v2(Bucket=bucket, Prefix=reviews_prefix)
reviews_parquet_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')]

if reviews_parquet_files:
    print(f"Reading {len(reviews_parquet_files)} parquet file(s) from S3...")
    
    for i, parquet_path in enumerate(reviews_parquet_files):
        full_s3_path = f"s3://{bucket}/{parquet_path}"
        print(f"\n[{i+1}/{len(reviews_parquet_files)}] Reading: {full_s3_path}")
        
        # Read parquet file from S3
        df_temp = pd.read_parquet(full_s3_path)
        print(f"  Shape: {df_temp.shape}")
        
        # Append to main dataframe
        df_all_reviews_s3 = pd.concat([df_all_reviews_s3, df_temp], ignore_index=True)
    
    print(f"\n{'='*60}")
    print(f"Combined DataFrame shape: {df_all_reviews_s3.shape}")
    print(f"Columns: {list(df_all_reviews_s3.columns)}")
    print(f"\nFirst few rows:")
    display(df_all_reviews_s3.head())
    
    print(f"\nDataFrame info:")
    df_all_reviews_s3.info()
else:
    print("No parquet files available to read.")

Reading 1 parquet file(s) from S3...

[1/1] Reading: s3://mlops-backblaze-d7b30cb5-us-east-1/raw/reviews_2023_parquet/raw_review_Electronics/part-000000.parquet
  Shape: (171573, 10)

Combined DataFrame shape: (171573, 10)
Columns: ['rating', 'title', 'text', 'images', 'asin', 'parent_asin', 'user_id', 'timestamp', 'helpful_vote', 'verified_purchase']

First few rows:


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,4.0,Great solid flash drive - Toshiba quality and ...,Not much can be said that hasn't been said in ...,[],B00IEZGU6G,B00WHEUS22,AFSKPY37N3C43SOI5IEXEK5JSIYA,1411650573000,0,False
1,5.0,GREAT External Drive,great price - I already had a Seagate 4TB an...,[],B01M00UHV8,B07454F4JH,AHGAOIZVODNHYMNCBV4DECZH42UQ,1494691925000,1,True
2,3.0,Long Term Life is Questionable; Seagate not go...,"The first one of these I got was DOA, but Seag...",[],B01IEKG3TY,B01IEKG3TY,AFAIJYOUO3NAWLBDIKTQSC3DASWA,1472494627000,1,False
3,4.0,"Compact Attractive, Quiet Unit",Several reviewers mentioned the difficulty in ...,[],B002HKEQZ6,B002OB4DC4,AFAIJYOUO3NAWLBDIKTQSC3DASWA,1306211962000,1,False
4,5.0,Works perfectly,I had a WD blue M.2 SSD that kept dropping off...,[],B09JCD2CN6,B09JCD2CN6,AFJBKPK5W56XWSNPQU2WW66ISWYQ,1640360340432,0,False



DataFrame info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171573 entries, 0 to 171572
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   rating             171573 non-null  float64
 1   title              171573 non-null  object 
 2   text               171573 non-null  object 
 3   images             171573 non-null  object 
 4   asin               171573 non-null  object 
 5   parent_asin        171573 non-null  object 
 6   user_id            171573 non-null  object 
 7   timestamp          171573 non-null  int64  
 8   helpful_vote       171573 non-null  int64  
 9   verified_purchase  171573 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 11.9+ MB


## Apply Join Columns - Extract Manufacturer and Model Information

In [19]:
import re

def extract_manufacturer(text):
    """Extract manufacturer from review text"""
    text_lower = str(text).lower()
    
    # Map various manufacturer names to standard format
    if 'toshiba' in text_lower:
        return 'TOSHIBA'
    elif 'seagate' in text_lower or 'st' in text_lower[:3]:
        return 'SEAGATE'
    elif 'western digital' in text_lower or 'wd' in text_lower or 'wdc' in text_lower:
        return 'WDC'
    elif 'hitachi' in text_lower or 'hgst' in text_lower:
        return 'HGST'
    
    return None

def extract_model_hints(text):
    """Extract potential model numbers/patterns from review text"""
    text_upper = str(text).upper()
    
    # Look for common model patterns
    patterns = [
        r'[A-Z]{2,}\s*[A-Z0-9]{6,}',  # e.g., "MG08ACA16TA", "WUH722222ALE6L4"
        r'ST\d{4,}[A-Z]{2}\d{3,}[A-Z]?',  # Seagate pattern
        r'WD[A-Z0-9]{6,}',  # WD pattern
        r'MG\d{2}[A-Z]{3}\d{2}[A-Z]{2,}',  # Toshiba pattern
    ]
    
    models = []
    for pattern in patterns:
        matches = re.findall(pattern, text_upper)
        models.extend(matches)
    
    return models if models else None

# Add manufacturer and model hints to the parquet reviews dataframe
print("Extracting manufacturer from review titles...")
df_all_reviews_s3['manufacturer'] = df_all_reviews_s3['title'].apply(extract_manufacturer)

print("Filling missing manufacturers from review text...")
df_all_reviews_s3['manufacturer'] = df_all_reviews_s3['manufacturer'].fillna(
    df_all_reviews_s3['text'].apply(extract_manufacturer)
)

print("Extracting model hints from titles...")
df_all_reviews_s3['model_hints'] = df_all_reviews_s3['title'].apply(extract_model_hints)

print("Filling missing model hints from review text...")
df_all_reviews_s3['model_hints'] = df_all_reviews_s3['model_hints'].fillna(
    df_all_reviews_s3['text'].apply(extract_model_hints)
)

print("\n" + "="*80)
print("Reviews with manufacturer extracted:")
print(df_all_reviews_s3[['parent_asin', 'title', 'manufacturer', 'model_hints']].head(10))
print(f"\nReviews with manufacturer identified: {df_all_reviews_s3['manufacturer'].notna().sum():,} / {len(df_all_reviews_s3):,}")

print("\n" + "="*80)
print("Manufacturer distribution in reviews:")
print(df_all_reviews_s3['manufacturer'].value_counts())

Extracting manufacturer from review titles...
Filling missing manufacturers from review text...
Extracting model hints from titles...
Filling missing model hints from review text...

Reviews with manufacturer extracted:
  parent_asin                                              title manufacturer  \
0  B00WHEUS22  Great solid flash drive - Toshiba quality and ...      TOSHIBA   
1  B07454F4JH                               GREAT External Drive      SEAGATE   
2  B01IEKG3TY  Long Term Life is Questionable; Seagate not go...      SEAGATE   
3  B002OB4DC4                     Compact Attractive, Quiet Unit      SEAGATE   
4  B09JCD2CN6                                    Works perfectly          WDC   
5  B0C6QYPNYR                               Great for mobile use          WDC   
6  B01FRP1ZHE                           Chic looking hard drive.          WDC   
7  B01BCWKBZI                                    Fast and small.          WDC   
8  B09VS4V18K                                      

## Analyze Low Rating Distribution by Manufacturer

In [20]:
# Filter to only reviews with identified manufacturers
df_with_mfr = df_all_reviews_s3[df_all_reviews_s3['manufacturer'].notna()].copy()

print(f"Analyzing {len(df_with_mfr):,} reviews with identified manufacturers\n")

# Group by manufacturer and calculate rating distributions
mfr_ratings = df_with_mfr.groupby('manufacturer')['rating'].agg([
    ('total_reviews', 'count'),
    ('one_star_count', lambda x: (x == 1.0).sum()),
    ('two_star_count', lambda x: (x == 2.0).sum()),
]).reset_index()

# Calculate percentages
mfr_ratings['pct_one_star'] = (mfr_ratings['one_star_count'] / mfr_ratings['total_reviews'] * 100).round(2)
mfr_ratings['pct_two_star'] = (mfr_ratings['two_star_count'] / mfr_ratings['total_reviews'] * 100).round(2)

# Create final dataframe with manufacturer as index
df_mfr_low_ratings = mfr_ratings[['manufacturer', 'pct_one_star', 'pct_two_star']].set_index('manufacturer')

print("="*80)
print("PERCENTAGE OF LOW RATINGS BY MANUFACTURER")
print("="*80)
print("\nDataFrame with manufacturer as index:")
print(df_mfr_low_ratings)

print("\n" + "="*80)
print("\nDetailed breakdown including counts:")
mfr_ratings_display = mfr_ratings.set_index('manufacturer')
print(mfr_ratings_display.sort_values('total_reviews', ascending=False))

print("\n" + "="*80)
print("\nKey insights:")
print(f"  • Manufacturer with highest % of 1-star reviews: {df_mfr_low_ratings['pct_one_star'].idxmax()} ({df_mfr_low_ratings['pct_one_star'].max()}%)")
print(f"  • Manufacturer with lowest % of 1-star reviews: {df_mfr_low_ratings['pct_one_star'].idxmin()} ({df_mfr_low_ratings['pct_one_star'].min()}%)")
print(f"  • Manufacturer with highest % of 2-star reviews: {df_mfr_low_ratings['pct_two_star'].idxmax()} ({df_mfr_low_ratings['pct_two_star'].max()}%)")
print(f"  • Manufacturer with lowest % of 2-star reviews: {df_mfr_low_ratings['pct_two_star'].idxmin()} ({df_mfr_low_ratings['pct_two_star'].min()}%)")

Analyzing 163,406 reviews with identified manufacturers

PERCENTAGE OF LOW RATINGS BY MANUFACTURER

DataFrame with manufacturer as index:
              pct_one_star  pct_two_star
manufacturer                            
HGST                 15.45          5.46
SEAGATE              25.12          7.27
TOSHIBA              17.01          6.10
WDC                  13.55          5.77


Detailed breakdown including counts:
              total_reviews  one_star_count  two_star_count  pct_one_star  \
manufacturer                                                                
WDC                  103758           14061            5985         13.55   
SEAGATE               40251           10111            2925         25.12   
TOSHIBA               16134            2744             984         17.01   
HGST                   3263             504             178         15.45   

              pct_two_star  
manufacturer                
WDC                   5.77  
SEAGATE               7.27 

## Join Review Features with Backblaze Data

In [21]:
# Fix manufacturer column in df_pandas to match standardized names
# Reapply the get_bb_manufacturer function to ensure consistency

def get_bb_manufacturer(model):
    """Extract manufacturer from Backblaze model name"""
    model_str = str(model)
    if model_str.startswith('TOSHIBA'):
        return 'TOSHIBA'
    elif model_str.startswith('ST') or model_str.startswith('SEAGATE'):
        return 'SEAGATE'
    elif model_str.startswith('WDC') or model_str.startswith('WD'):
        return 'WDC'
    elif model_str.startswith('HGST') or model_str.startswith('HITACHI'):
        return 'HGST'
    return None

# Re-create manufacturer column with standardized names
df_pandas['manufacturer'] = df_pandas['model'].apply(get_bb_manufacturer)

print("Standardized manufacturer distribution in df_pandas:")
print(df_pandas['manufacturer'].value_counts())

# Join the manufacturer rating features with the Backblaze data
mfr_ratings_for_join = mfr_ratings[['manufacturer', 'pct_one_star', 'pct_two_star', 'total_reviews', 
                                     'one_star_count', 'two_star_count']].copy()

# Perform the join on manufacturer column
df_joined = df_pandas.merge(
    mfr_ratings_for_join, 
    on='manufacturer', 
    how='left',
    suffixes=('', '_reviews')
)

print("\n" + "="*80)
print("JOINED DATAFRAME: BACKBLAZE DATA + REVIEW FEATURES")
print("="*80)
print(f"\nOriginal df_pandas shape: {df_pandas.shape}")
print(f"Joined df_joined shape: {df_joined.shape}")
print(f"New columns added: {df_joined.shape[1] - df_pandas.shape[1]}")

print("\n" + "="*80)
print("New review-based features added to each Backblaze record:")
print("  • total_reviews - Total number of reviews for this manufacturer")
print("  • one_star_count - Count of 1-star reviews for this manufacturer")
print("  • two_star_count - Count of 2-star reviews for this manufacturer")
print("  • pct_one_star - Percentage of 1-star reviews (quality indicator)")
print("  • pct_two_star - Percentage of 2-star reviews (quality indicator)")

print("\n" + "="*80)
print("Sample rows with key columns:")
sample_cols = ['model', 'manufacturer', 'failure', 'pct_one_star', 'pct_two_star', 
               'total_reviews', 'capacity_bytes']
# Only show columns that exist
sample_cols_exist = [col for col in sample_cols if col in df_joined.columns]
display(df_joined[sample_cols_exist].head(10))

print("\n" + "="*80)
print("Review feature statistics by manufacturer:")
display(df_joined.groupby('manufacturer')[['pct_one_star', 'pct_two_star', 'total_reviews']].first())

print("\n" + "="*80)
print("Checking for missing review features (manufacturers without reviews):")
missing_reviews = df_joined['pct_one_star'].isna().sum()
print(f"Records without review features: {missing_reviews:,} / {len(df_joined):,}")
if missing_reviews > 0:
    print("\nManufacturers with missing review data:")
    print(df_joined[df_joined['pct_one_star'].isna()]['manufacturer'].value_counts())

Standardized manufacturer distribution in df_pandas:
manufacturer
SEAGATE    559901
TOSHIBA    549909
WDC        383791
HGST       136048
Name: count, dtype: int64

JOINED DATAFRAME: BACKBLAZE DATA + REVIEW FEATURES

Original df_pandas shape: (1645930, 201)
Joined df_joined shape: (1645930, 206)
New columns added: 5

New review-based features added to each Backblaze record:
  • total_reviews - Total number of reviews for this manufacturer
  • one_star_count - Count of 1-star reviews for this manufacturer
  • two_star_count - Count of 2-star reviews for this manufacturer
  • pct_one_star - Percentage of 1-star reviews (quality indicator)
  • pct_two_star - Percentage of 2-star reviews (quality indicator)

Sample rows with key columns:


,model,manufacturer,failure,pct_one_star,pct_two_star,total_reviews,capacity_bytes
0,CT250MX500SSD1,None,0,NaN,NaN,NaN,250059350016
1,CT250MX500SSD1,None,0,NaN,NaN,NaN,250059350016
2,CT250MX500SSD1,None,0,NaN,NaN,NaN,250059350016
3,CT250MX500SSD1,None,0,NaN,NaN,NaN,250059350016
4,CT250MX500SSD1,None,0,NaN,NaN,NaN,250059350016
5,HGST HUH728080ALE604,HGST,0,15.45,5.46,3263.0,8001563222016
6,HGST HUH728080ALE604,HGST,0,15.45,5.46,3263.0,8001563222016
7,HGST HUH728080ALE604,HGST,0,15.45,5.46,3263.0,8001563222016
8,Seagate BarraCuda SSD ZA250CM10002,None,0,NaN,NaN,NaN,250059350016
9,ST500LM012 HN,SEAGATE,0,25.12,7.27,40251.0,500107862016



Review feature statistics by manufacturer:


,pct_one_star,pct_two_star,total_reviews
manufacturer,,,
HGST,15.45,5.46,3263.0
SEAGATE,25.12,7.27,40251.0
TOSHIBA,17.01,6.10,16134.0
WDC,13.55,5.77,103758.0



Checking for missing review features (manufacturers without reviews):
Records without review features: 16,281 / 1,645,930

Manufacturers with missing review data:
Series([], Name: count, dtype: int64)
